In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.text import Tokenizer
import gc                         
gc.enable()
import tensorflow as tf
from keras.regularizers import l1, l2
from keras.layers import Embedding, Input, Dense, Reshape, Flatten, Dropout, Concatenate, Multiply
from sklearn.preprocessing import StandardScaler 
scale_features_std = StandardScaler() 

In [61]:
class PREDICTION_NN(tf.keras.Model):
  
  def __init__(self):
        
        """
        
        """

        super().__init__()

  def prepare_data(path_to_file, sep_type, diff_days):
      print('Загружаем данные из файла')
      df_transactions = pd.read_csv(path_to_file, sep=sep_type)#для работы в Colab df = pd.read_csv("'/content/drive/MyDrive/transactions.csv', sep=','")

      print('Формируем датафрейм с максимальным количеством заказов для каждого пользователя') 
      df_user = df_transactions.groupby('user_id')[['order_number']].max()
      df_user.columns = ['user_total_orders']
      df_user = df_user.reset_index()

      print('Формируем вспомогательный датафрейм с деталями заказов')
      df_orders = df_transactions[['user_id', 'order_id', 'product_id', 'add_to_cart_order', 'reordered']]

      print("Вычисляем среднее по столбцу 'reordered' для каждого пользователя и формируем датафрейм")
      df_user_reorder = df_transactions.groupby('user_id')['reordered'].mean().to_frame('user_reordered_ratio').reset_index()

      print("Вычисляем среднее значение позиции товара в чеке по каждой паре 'пользователь/товар', затем сортируем по возрастанию")
      df_add_to_cart_order = df_transactions.groupby(['user_id', 'product_id'])['add_to_cart_order'].mean().to_frame('user_add_to_cart_order_ratio').reset_index()
      df_add_to_cart_order = df_add_to_cart_order.sort_values(by=['user_id', 'user_add_to_cart_order_ratio'])

      print('Объединяем df_transactions и df_user')
      df_transactions = pd.merge(df_transactions, df_user, on=['user_id'], how='left')
      
      print("Добавляем столбец 'diff', представляющий собой разницу между общим кол-вом заказов и текущим номером заказа")
      df_transactions['diff'] = df_transactions['user_total_orders'] - df_transactions['order_number']

      print('Объединяем df_user_reorder и df_user')
      df_user = df_user.merge(df_user_reorder, on='user_id', how='left')

      del df_user_reorder
      gc.collect()

      print('Находим общее количество приобретенных товаров по каждому product_id	и формируем датафрейм')
      df_purchases = df_transactions.groupby('product_id')['order_id'].count().to_frame('product_total_purchases').reset_index()

      print("Вычисляем среднее по столбцу 'reordered' для каждого товара и формируем датафрейм")
      df_product_reorder = df_transactions.groupby('product_id')['reordered'].mean().to_frame('product_reorder_ratio').reset_index()

      print('Объединяем df_purchases и df_product_reorder')
      df_purchases = df_purchases.merge(df_product_reorder, on='product_id', how='left')

      del df_product_reorder
      gc.collect()

      print("Удаляем Nan из столбца 'product_reorder_ratio'")
      df_purchases['product_reorder_ratio'] = df_purchases['product_reorder_ratio'].fillna(value=0)

      print("Находим общее количество товаров по каждой паре 'пользователь/товар', сортируем столбец по убыванию 'user_product_total_purchases'")
      df_user_product_purchases = df_transactions.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('user_product_total_purchases').reset_index()
      df_user_product_purchases = df_user_product_purchases.sort_values(by=['user_id', 'user_product_total_purchases'], ascending = [True, False])

      df_total_orders = df_transactions.groupby('user_id')['order_number'].max().to_frame('total_orders')

      print("Формируем датафрейм с номерами заказов, в которых впервые встречается каждая пара 'пользователь/товар'")
      df_first_order = df_transactions.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number').reset_index()

      print('Формируем датафрейм как объединение df_total_orders и df_first_order')
      df_span = pd.merge(df_total_orders, df_first_order, on='user_id', how='right')

      print("Добавляем столбец 'order_range_denominator', представляющий разность между общим количеством заказов и первым заказом")
      print("по каждой паре 'пользователь/товар', сортируем 'order_range_denominator' по убыванию")
      df_span['order_range_denominator'] = df_span['total_orders'] - df_span['first_order_number'] + 1
      df_span = df_span.sort_values(by=['user_id', 'order_range_denominator'], ascending = [True, False])

      print('Объединяем df_user_product_purchases и df_span')
      df_user_product_ratio = pd.merge(df_user_product_purchases, df_span, on=['user_id', 'product_id'], how='left')

      print("Добавляем столбец 'user_product_reorder_ratio', представляющий собой отношение")
      print("общего кол-ва покупок к деноминатору заказа по каждой паре 'пользователь/товар'")
      print("Сортируем 'user_product_reorder_ratio' по убыванию")
      df_user_product_ratio['user_product_reorder_ratio'] = df_user_product_ratio['user_product_total_purchases'] / df_user_product_ratio['order_range_denominator']
      df_user_product_ratio = df_user_product_ratio.sort_values(by=['user_id', 'user_product_reorder_ratio'], ascending=[True, False])

      del df_user_product_ratio['user_product_total_purchases']
      del df_user_product_ratio['total_orders']
      del df_user_product_ratio['first_order_number']
      del df_user_product_ratio['order_range_denominator']
      del df_span
      del df_first_order
      gc.collect()

      print("Объединяем df_user_product_ratio и df_user_product_purchases")
      print("Сортируем 'user_product_reorder_ratio' по убыванию")
      df_user_product_purchases = df_user_product_purchases.merge(df_user_product_ratio, on=['user_id', 'product_id'], how='left')
      df_user_product_purchases = df_user_product_purchases.sort_values(by=['user_id', 'user_product_reorder_ratio'], ascending=[True, False])

      del df_user_product_ratio
      gc.collect()

      print("Добавляем столбец 'back_order_no', который представляет собой разницу между максимальным значением по столбцу 'order_number'") 
      print("для каждого пользователя и текущим номером заказа")
      df_transactions['back_order_no'] = df_transactions.groupby('user_id')['order_number'].transform(max) - df_transactions['order_number'] + 1

      print("Формируем датафрейм, состоящий только из последних пяти заказов каждого пользователя")
      df_transactions_5 = df_transactions.loc[df_transactions['back_order_no'] <= 5]

      del df_transactions
      gc.collect()

      print("Создаем тренировочный и тестовый датасеты")
      print("train представляет собой заказы всех пользователей за исключением заказов последних 'diff_days'")
      print("test - заказы всех пользователей в последние 'diff_days'")
      #рекомендованное значение diff_days = 1
      train = df_transactions_5.loc[df_transactions_5['diff']>diff_days]
      test = df_transactions_5.loc[df_transactions_5['diff']<=diff_days]

      print("Формируем датафрейм с количеством заказов, в которых присутствовали определенные товары в последних пяти заказах")
      orders_5_train = train.groupby(['user_id','product_id'])[['order_id']].count()
      orders_5_train.columns = ['last_5']

      print("Объединяем orders_5_train с датафреймом, представляющим только те товары, которые перезаказывались")
      orders_6_train = df_transactions_5.groupby(['user_id','product_id'])[['reordered']].max()
      orders_5_train = orders_5_train.merge(orders_6_train, on=['user_id', 'product_id'], how='left')

      del orders_6_train
      gc.collect()

      print("Объединяем df_user_product_purchases и orders_5_train")
      df_user_product_purchases_train = df_user_product_purchases.merge(orders_5_train, on=['user_id', 'product_id'], how='inner')

      print("Объединяем df_user_product_purchases_train и df_user")
      train = df_user_product_purchases_train.merge(df_user, on='user_id', how='inner')

      print("Объединяем train и df_purchases")
      train = train.merge(df_purchases, on='product_id', how='inner')

      print("Формируем датафрейм с количеством заказов, в которых присутствовали определенные товары в последних пяти заказах")
      orders_5_test = test.groupby(['user_id','product_id'])[['order_id']].count()
      orders_5_test.columns = ['last_5']

      print("Объединяем orders_5_test с датафреймом, представляющим только те товары, которые перезаказывались")
      orders_6_test = df_transactions_5.groupby(['user_id','product_id'])[['reordered']].max()
      orders_5_test = orders_5_test.merge(orders_6_test, on=['user_id', 'product_id'], how='left')

      del orders_6_test
      gc.collect()

      print("Объединяем df_user_product_purchases и orders_5_test")
      df_user_product_purchases_test = df_user_product_purchases.merge(orders_5_test, on=['user_id', 'product_id'], how='inner')

      print("Объединяем df_user_product_purchases_test и df_user")
      test = df_user_product_purchases_test.merge(df_user, on='user_id', how='inner')
      test = test.merge(df_purchases, on='product_id', how='inner')

      del df_user
      del df_purchases
      del df_user_product_purchases
      gc.collect()

      print('Формируем x_train, y_train, x_test, y_test')
      y_train = train['reordered'].ravel()#target тренировочный
      x_train = train.drop(['reordered'], axis=1)#признаки без target тренировочные
      y_test = test['reordered'].ravel()#target тестовый
      x_test = test.drop(['reordered'], axis=1)#признаки без target тестовые

      return x_train, y_train, x_test, y_test
  
  def preprocess_data(x_train, x_test, scale_features_std):
    #scale_features_std = StandardScaler()
    user_train = x_train[['user_id', 'user_total_orders', 'user_reordered_ratio']]
    scaler = scale_features_std.fit(user_train)
    user_train = scale_features_std.transform(user_train)

    product_train = x_train[['product_id', 'user_product_total_purchases', 'user_product_reorder_ratio', 'last_5', 'product_total_purchases', 'product_reorder_ratio']]
    scaler = scale_features_std.fit(product_train)
    product_train = scale_features_std.transform(product_train)
    
    user_test = x_test[['user_id', 'user_total_orders', 'user_reordered_ratio']]
    scaler = scale_features_std.fit(user_test)
    user_test = scale_features_std.transform(user_test)
    
    product_test = x_test[['product_id', 'user_product_total_purchases', 'user_product_reorder_ratio', 'last_5', 'product_total_purchases', 'product_reorder_ratio']]
    scaler = scale_features_std.fit(product_test)
    product_test = scale_features_std.transform(product_test)
    return user_train, product_train, user_test, product_test
  
  def create_model(layers, reg_layers, user_n_factors, product_n_factors):
    #например, layers = [20, 10]
    #например, reg_layers=[0, 0]
    assert len(layers) == len(reg_layers)
    num_layer = len(layers)
      
    user_input = tf.keras.Input(shape=(user_n_factors,), dtype='int64', name='user')
    product_input = tf.keras.Input(shape=(product_n_factors,), dtype='int64', name='item')
      
    user_latent = tf.keras.layers.Flatten()(user_input)
    product_latent = tf.keras.layers.Flatten()(product_input)
      
    vector = Concatenate()([user_latent, product_latent])
      
    for idx in range(1, num_layer):
      layer = Dense(layers[idx], kernel_regularizer = l2(reg_layers[idx]), activation='relu', name = 'layer%d' %idx)
      vector = layer(vector)
      
    prediction = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(vector)
      
    model = tf.keras.models.Model(inputs = [user_input, product_input], outputs = prediction)
      
    return model
    
  def compile_model(model, optimizer, loss):
    #optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = tf.keras.losses.MSE
    model.compile(optimizer=optimizer, loss=loss, metrics=['mse'])
    model.summary()

    #функция предназначена для создания датафрейма из тестового набора данных и списка полученных предсказаний
  def create_predicted_df(users, products, predictions, columns=['user_id', 'product_id', 'predictions']):
    predicted_df = pd.DataFrame(list(zip(users, products, predictions)), columns=columns)
    def extract_value(array):
      return array[0]
    predicted_df['predictions'] = predicted_df['predictions'].apply(extract_value)
    predicted_df = predicted_df.sort_values(by=['user_id', 'predictions'], ascending=[True, False])
    return predicted_df

  def clean_prediction(row):
    data = row.product_id
    data = str("".join(str(data))[1:-1].replace(',', ' '))
    return data    
  
  def get_recommendations(predicted_df):
    rec = predicted_df.groupby('user_id')['product_id'].apply(list).reset_index(name='product_id')
    #функция предназначена для удаления нулевых элементов из списка,
    #а также для формирования срезов из 10-ти элементов      
    def fetch_ten(l):
      for i in l:
        if i == 0:
          l.remove(i)
        elif len(l)>10:
          return l[:10]
        else:
          return l
    rec['product_id'] = rec['product_id'].apply(fetch_ten)
    #функция предназначена для приведения строк к формату, который требуется в условии задачи
    def clean_prediction(row):
      data = row.product_id
      data = str("".join(str(data))[1:-1].replace(',', ' '))
      return data
    rec['product_id'] = rec.apply(clean_prediction, axis=1)
    return rec
    
  def get_recommendations_for_user(path_to_submission_file, path_to_products_file, user_id):
    final_rec = pd.read_csv(path_to_submission_file)
    products = pd.read_csv(path_to_products_file)
    rec_for_user = final_rec.loc[final_rec['user_id'] == user_id]
    s = (rec_for_user['product_id'].to_list())[0].split()
    for i in s:
      for index, row in products.iterrows():
        if row['product_id'] == int(i):
          print(row['product_name'])

In [20]:
x_train_df, y_train_df, x_test_df, y_test_df = PREDICTION_NN.prepare_data('/content/drive/MyDrive/transactions.csv', ',', 1)

Загружаем данные из файла
Формируем датафрейм с максимальным количеством заказов для каждого пользователя
Формируем вспомогательный датафрейм с деталями заказов
Вычисляем среднее по столбцу 'reordered' для каждого пользователя и формируем датафрейм
Вычисляем среднее значение позиции товара в чеке по каждой паре 'пользователь/товар', затем сортируем по возрастанию
Объединяем df_transactions и df_user
Добавляем столбец 'diff', представляющий собой разницу между общим кол-вом заказов и текущим номером заказа
Объединяем df_user_reorder и df_user
Находим общее количество приобретенных товаров по каждому product_id	и формируем датафрейм
Вычисляем среднее по столбцу 'reordered' для каждого товара и формируем датафрейм
Объединяем df_purchases и df_product_reorder
Удаляем Nan из столбца 'product_reorder_ratio'
Находим общее количество товаров по каждой паре 'пользователь/товар', сортируем столбец по убыванию 'user_product_total_purchases'
Формируем датафрейм с номерами заказов, в которых впервы

In [23]:
user_train_df, product_train_df, user_test_df, product_test_df = PREDICTION_NN.preprocess_data(x_train_df, x_test_df, StandardScaler())

In [28]:
rec_model = PREDICTION_NN.create_model([20, 10], [0, 0], 3, 6)

In [32]:
PREDICTION_NN.compile_model(rec_model, tf.keras.optimizers.Adam(learning_rate=1e-3), tf.keras.losses.MSE)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 3)]          0                                            
__________________________________________________________________________________________________
item (InputLayer)               [(None, 6)]          0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 3)            0           user[0][0]                       
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 6)            0           item[0][0]                       
______________________________________________________________________________________________

In [33]:
hist = rec_model.fit([user_train_df, product_train_df], #input
                 y_train_df, # labels
                 batch_size=16, epochs=50, steps_per_epoch=10, shuffle=True, validation_split=0.3)

Epoch 1/50
10/10 [==============================] - 33s 4s/step - loss: 0.2261 - mse: 0.2261 - val_loss: 0.2259 - val_mse: 0.2259
Epoch 2/50
10/10 [==============================] - 32s 4s/step - loss: 0.2207 - mse: 0.2207 - val_loss: 0.2221 - val_mse: 0.2221
Epoch 3/50
10/10 [==============================] - 33s 4s/step - loss: 0.2164 - mse: 0.2164 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 4/50
10/10 [==============================] - 32s 4s/step - loss: 0.2159 - mse: 0.2159 - val_loss: 0.2158 - val_mse: 0.2158
Epoch 5/50
10/10 [==============================] - 32s 4s/step - loss: 0.2139 - mse: 0.2139 - val_loss: 0.2134 - val_mse: 0.2134
Epoch 6/50
10/10 [==============================] - 33s 4s/step - loss: 0.1963 - mse: 0.1963 - val_loss: 0.2111 - val_mse: 0.2111
Epoch 7/50
10/10 [==============================] - 32s 4s/step - loss: 0.2104 - mse: 0.2104 - val_loss: 0.2089 - val_mse: 0.2089
Epoch 8/50
10/10 [==============================] - 33s 4s/step - loss: 0.2031 - mse: 0.20

In [34]:
preds = rec_model.predict([user_test_df, product_test_df])

In [37]:
rec_df = PREDICTION_NN.create_predicted_df(x_test_df['user_id'], x_test_df['product_id'], preds, columns=['user_id', 'product_id', 'predictions'])

In [43]:
rec_df

,user_id,product_id,predictions
2519,1,46149,0.976993
0,1,196,0.964896
1714,1,12427,0.953206
2172,1,25133,0.953206
2021,1,10258,0.942491
...,...,...,...
1386737,206209,6846,0.593171
1548399,206209,5622,0.593171
120183,206209,14197,0.516883
1294603,206209,20590,0.434173


In [51]:
final_rec = PREDICTION_NN.get_recommendations(rec_df)

In [52]:
final_rec

,user_id,product_id
0,1,46149 196 12427 25133 10258 35951 38928 ...
1,2,24852 47209 7963 21709 33754 19057 20785...
2,3,47766 39190 17668 21903 43961 1005 32402...
3,7,21137 47272 40852 37602 37999 27690 2999...
4,13,27086 4210 43086 27435 42248 41351 5025 ...
...,...,...
99995,206202,12919 38837 24852 17038 17459 41177 432 ...
99996,206206,27086 11520 29326 13045 45681 38739 1689...
99997,206207,13176 33754 36011 39180 44632 33787 2734...
99998,206208,13176 34213 21137 47626 27845 23579 2099...


In [54]:
#формируем файл с рекомендациями
submission_9 = final_rec.to_csv('submission_9.csv', index=False)

In [62]:
preds_for_13 = PREDICTION_NN.get_recommendations_for_user('/content/drive/MyDrive/submission_9.csv', '/content/drive/MyDrive/products.csv', 13)

Half & Half
Whole Milk
Super Greens Salad
Whole Wheat Pita Bread Loaves
Coconut Milk, Classic
Family Size Naturally Flavored Whole Grain Oats Cereal
Green Onions
Bok Choy
Cilantro Bunch
Tomato Paste
